In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('../..')
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings
from langchain_core.messages import HumanMessage

## UserInputLLM

### utils

In [23]:
import json
from typing import Any, Dict, Iterator, List, Optional, Union
from langchain_core.messages import AIMessage

def _make_tools_invocation(name_to_arguments: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    tool_calls = [
        {"function": {"name": name, "arguments": json.dumps(arguments)}, "id": idx}
        for idx, (name, arguments) in enumerate(name_to_arguments.items())
    ]

    return {"tool_calls": tool_calls}

def get_input(prompt: str) -> str:
    return input(prompt)


### Tool

In [22]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def confirm(topic: str) -> str:
    """确认来自LLM的Topic或Content创作结果"""
    return topic

### LLM

In [18]:
import time
import re
from typing import Any, Dict, Iterator, List, Optional, Union
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, HumanMessageChunk, AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

class ChatWithUser(BaseChatModel):
    """允许用户参与的大模型"""

    @property
    def _llm_type(self) -> str:
        return "chat-with-user"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        generations = [ChatGeneration(message=res) for res in self._ask(messages)]
        return ChatResult(generations=generations)

    def _ask(self, messages: List[BaseMessage]) -> List[BaseMessage]:
        user_input = get_input("👤: ")
        if user_input == "ok":
            tool_resp = _make_tools_invocation({"confirm": "{'topic': 'my_topic'}"})
            response = AIMessage(content="", additional_kwargs=tool_resp)
        else:
            response = AIMessage(user_input)            
        return [response]


In [20]:
u = ChatWithUser()
u.invoke("hello?")

AIMessage(content='hi', id='run-b4f0d6eb-04a9-4ad5-8bf8-0dfd50d0a0da-0')

## Topic Chain

## Content Chain

## Graph

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, MessageGraph, END
import operator

# 定义业务逻辑
def nodeA(state):
    return ["AAA"]

from langchain.llms.fake import FakeStreamingListLLM
llm = FakeStreamingListLLM(responses=["你好，我是一个模拟的大语言模型"])

# 定义图
workflow = MessageGraph()

workflow.add_node("a", nodeA)
workflow.add_node("b", llm)

workflow.add_edge("a", "b")
workflow.add_edge("b", END)

workflow.set_entry_point("a")

app = workflow.compile()

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

from langchain_chinese import (
    AskDocumentTool,
    create_qa_chain,
    create_reason_agent,
)
from langchain_chinese.langgraph import create_tools_calling_executor

query_document = Document(page_content="langchain_chinese 是为中国大语言模型优化的langchain模块")

db = Chroma.from_documents([query_document], ZhipuAIEmbeddings())

llm_zhipu = ChatZhipuAI(model="glm-4")

chain = create_qa_chain(llm_zhipu, db.as_retriever())
qa_tool = AskDocumentTool(qa_chain=(chain | StrOutputParser()), name="qa_chain")

In [17]:
chain.get_graph().print_ascii()

              +---------------------------------+          
              | Parallel<context,question>Input |          
              +---------------------------------+          
                    ****               ****                
                 ***                       ***             
               **                             ***          
    +-------------+                              **        
    | Lambda(...) |                               *        
    +-------------+                               *        
            *                                     *        
            *                                     *        
            *                                     *        
+----------------------+                          *        
| VectorStoreRetriever |                          *        
+----------------------+                          *        
            *                                     *        
            *                           

In [18]:
chain.invoke(["langchain_chinese 是啥"])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AIMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化的langchain模块。它可能是用于提升语言模型在特定应用场景下的表现，比如在中文语境下的理解、生成和推理能力。这个模块的设计目的是为了更好地适应中文语言特点，提高语言模型在中国市场的应用效率和质量。')

In [3]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [4]:
# from langchain_openai import ChatOpenAI
# llm_zhipu = ChatZhipuAI()
tools = [qa_tool, where_is_cat_hiding]
app = create_tools_calling_executor(llm_zhipu, tools)

In [5]:
from langchain_core.messages import HumanMessage
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    verbose = True)

inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])        

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(



 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call tool [qa_chain]
{}


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'chunk': [ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提升语言模型在各种应用场景下的性能和效率，更好地适应中文语言环境的特点。', tool_call_id='call_8483741731860767639')]}
{'input': ['请查询资料，告诉我langchain_chinese是什么？', AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8483741731860767639', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'qa_chain'}, 'type': 'function'}]})], 'output': [ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提升语言模型在各种应用场景下的性能和效率，更好地适应中文语言环境的特点。', tool_call_id='call_8483741731860767639')]}

 on_chat_model_start ChatZhipuAI ['seq:step:2']
根据_我的_查询_结果_，_lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优化_设计的_lang_chain_模块_。_这个_模块_可能是_用于_提升_语言_模型_在各种_应用_场景_下的_性能_和_效率_，_更好地_适应_中文_语言_环境_的特点_。__

In [21]:
app.get_graph().print_ascii()

                    +-----------+             
                    | __start__ |             
                    +-----------+             
                          *                   
                          *                   
                          *                   
                      +-------+               
                      | agent |               
                     *+-------+*              
                   **           ***           
                 **                **         
               **                    **       
+-----------------------+              **     
| agent_should_continue |               *     
+-----------------------+               *     
            *           *****           *     
            *                ****       *     
            *                    ***    *     
       +---------+                +--------+  
       | __end__ |                | action |  
       +---------+                +--------+  


In [7]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": chain},
    verbose = True)

inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])


 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call runnable [qa_chain]


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



 on_chat_model_start ChatZhipuAI ['seq:step:4']
lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优_化的_ lang_chain_ 模_块_。_它_可能是_ lang_chain_ 库_的一个_分支_或_扩展_，_旨在_更好地_适应_中文_语境_和_中国的_相关_技术_需求_。_具体的_细节_需要_查阅_更多_关于_ lang_chain__ch_inese_ 的_官方_文档_或_资料_才能_了解_。__

In [23]:
app.get_graph().print_ascii()

                        +-----------+                     
                        | __start__ |                     
                        +-----------+                     
                              *                           
                              *                           
                              *                           
                          +-------+                       
                          | agent |*                      
                         *+-------+ ****                  
                       **               ***               
                     **                    ***            
                   **                         ****        
    +-----------------------+                     **      
    | agent_should_continue |                      *      
    +-----------------------+**                    *      
           **        **        *******             *      
         **            **             *****        *    

In [8]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": {"node": chain, "to": "agent"}},
    verbose = True)
app.get_graph().print_ascii()

                              +-----------+                          
                              | __start__ |                          
                              +-----------+                          
                                     *                               
                                     *                               
                                     *                               
                                +-------+                            
                               *| agent |***                         
                            *** +-------+   ***                      
                       *****          *        *****                 
                    ***                *            ***              
                 ***                   *               *****         
+-----------------------+               *                   **       
| agent_should_continue |*             *                     *       
+-------------------

In [9]:
inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])


 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call runnable [qa_chain]


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



 on_chat_model_start ChatZhipuAI ['seq:step:4']
lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优化_和_改进_的_lang_chain_模块_。_它_可能是_lang_chain_这一_框架_的_扩展_或_衍生_版本_，_旨在_更好地_适应_中文_语言_环境_下的_需求_，_提升_大_语言_模型_在_中文_处理_方面的_性能_和_效果_。__
 on_chat_model_start ChatZhipuAI ['seq:step:2']
根据_我的_查询_结果_，_lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优化_和_改进_的_ lang_chain_ 模_块_。_它是_ lang_chain_ 这一_框架_的_扩展_或_衍生_版本_，_旨在_更好地_适应_中文_语言_环境_下的_需求_，_提升_大_语言_模型_在_中文_处理_方面的_性能_和_效果_。__

In [10]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  WhereIsCatHidding
log: call tool [WhereIsCatHidding]
{}
{'chunk': [ToolMessage(content='在床底下', tool_call_id='call_8483752349020160968')]}
{'input': [HumanMessage(content='猫在哪里？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8483752349020160968', 'function': {'arguments': '{"idea":"猫在哪里"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})], 'output': [ToolMessage(content='在床底下', tool_call_id='call_8483752349020160968')]}
根据_我的_观察_和_经验_，_猫_通常会_躲_在一些_狭_小_的地方_，_比如_床_底下_、_沙发_后面_或者_柜_子里_。_所以_，_如果你想_找到_一只_猫_，_最好_在这些_地方_仔细_寻找_。__

In [25]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  qa_chain
log: call runnable [qa_chain]


In [11]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

为什么_程序员_总是_携带_电脑_？_因为他们_不想_被人_称为_“_裸_奔_者_”。__